In [ ]:
import logging, sys
logging.disable(sys.maxsize)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# from dotenv import load_dotenv
# dotenv_ld = load_dotenv(verbose=False, override=True, interpolate=True)
import os
from pathlib import Path
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import display_markdown
from maulibs.GSheet import GSheet
from sklearn.preprocessing import StandardScaler
alt_dmr = alt.data_transformers.disable_max_rows()

In [ ]:
# with open('mmm_eda_ed_config.txt', 'r') as f:
#     input_gsheet_url = f.read()
input_gsheet_url = os.environ['GSHEET_URL']

input_gs_client = GSheet(input_gsheet_url)
input_title = input_gs_client.spreadsheet.title
feature_col_df = input_gs_client.read_dataframe('feature_list', header_row_num=0)
model_df = input_gs_client.read_dataframe('model_data', header_row_num=0)
model_df = model_df.reset_index(drop=True)
model_df['date'] = pd.to_datetime(model_df['date'])
features_lst = list(feature_col_df['feature'].values)
model_df['year'] = model_df['date'].dt.year.astype('str')
years = list(np.sort(model_df['year'].unique()))
year_options = ['all'] + years

for feature in features_lst + ['n_conversion']:
    model_df[feature] = model_df[feature].astype('float')


display_markdown(f'# Charts for [{input_title}]({input_gsheet_url}) ', raw=True)

## Channel Correlation with n_conversions

In [ ]:
for feature_name in features_lst:
    display_markdown(f'### {feature_name}', raw=True)
    
    corr_res = []
    for year in years:
        corr_res.append((year, model_df.loc[model_df['year'] == year, [feature_name, 'n_conversion']].corr()['n_conversion'][0]))
    
    corr_res.append(('total', model_df[[feature_name, 'n_conversion']].corr()['n_conversion'][0]))
    
    corr_df = pd.DataFrame(corr_res, columns=['year', 'corr']).set_index('year').T
    display_markdown(f'**correlation by year**', raw=True)
    display(corr_df.style.format({i: '{:,.0%}' for i in corr_df.columns}))
    
    base = alt.Chart(model_df).encode(alt.X('date'))
    
    feature_axis = base.mark_line(stroke='darkslateblue', interpolate='monotone').encode(
        alt.Y(feature_name).axis(labelColor='darkslateblue', titleColor='darkslateblue'))
    
    target_axis = base.mark_line(stroke='darkseagreen', interpolate='monotone').encode(
        alt.Y('n_conversion').axis(labelColor='darkseagreen', titleColor='darkseagreen'))
    
    feature_target_chart = alt.layer(feature_axis, target_axis).resolve_scale(y='independent').properties(
        title=f'{feature_name} vs. n_conversion', width=1200, height=600)
    
    display(feature_target_chart.interactive())

    display_markdown(f'---', raw=True)

In [ ]:

corr_dfs = []
for year in year_options:
    if year == 'all':
        corr_df = model_df[features_lst].corr().stack()
    else:
        corr_df = model_df.loc[model_df['year'] == year, features_lst].corr().stack()
    corr_df.index.names = ['feature_x', 'feature_y']
    corr_df = corr_df.reset_index().rename(columns={0: 'corr'})
    corr_df['corr_lab'] = 100 * corr_df['corr']
    corr_df['corr_lab'] = corr_df['corr_lab'].map('{:.0f}'.format)
    corr_df['year'] = year
    corr_dfs.append(corr_df)

corr_df = pd.concat(corr_dfs, axis=0, ignore_index=True)
copy_model_df = model_df.copy(deep=True)
copy_model_df['year'] = 'all'
copy_model_df = pd.concat([copy_model_df, model_df], axis=0, ignore_index=True)



In [ ]:
year_dropdown_corr_scat = alt.binding_select(options=year_options, name='year: ')
year_selection_corr_scat = alt.selection_point(fields=['year'], bind=year_dropdown_corr_scat, value='all')
# var_sel_cor = alt.selection_point(fields=['feature_x', 'feature_y'], clear=False, value='affiliate_mel_spend')
feature_x_selection_corr_scat = alt.selection_point(fields=['feature_x'], clear=False, name='feature_x_selection_corr_scat', value='sea_brand_mel_spend')
feature_y_selection_corr_scat = alt.selection_point(fields=['feature_y'], clear=False, name='feature_y_selection_corr_scat', value='affiliate_mel_spend')

corr_scat_base = alt.Chart(corr_df, title='Click a tile to compare').encode(
    x='feature_x', y='feature_y'    
).mark_rect().encode(
    color=alt.condition(feature_x_selection_corr_scat & feature_y_selection_corr_scat, alt.value('pink'), 'corr')
).add_params(
    year_selection_corr_scat, 
    feature_x_selection_corr_scat, 
    feature_y_selection_corr_scat
).transform_filter(
    year_selection_corr_scat
)

corr_scat_text = corr_scat_base.mark_text().encode(
    text='corr_lab', 
    color=alt.condition(alt.datum.corr > 0.5, alt.value('white'), alt.value('black'))
)

corr_scat_plot = corr_scat_base + corr_scat_text

scat_corr_plot = alt.Chart(copy_model_df).mark_point().encode(
    x=alt.X('x:Q', title='feature_x'), y=alt.Y('y:Q', title='feature_y')
).add_params(
    year_selection_corr_scat, 
    feature_x_selection_corr_scat, 
    feature_y_selection_corr_scat
).transform_calculate(
    x=f'datum[{feature_x_selection_corr_scat["feature_x"]}]', 
    y=f'datum[{feature_y_selection_corr_scat["feature_y"]}]'
).transform_filter(
    year_selection_corr_scat
)

display(alt.hconcat(corr_scat_plot.properties(width=600, height=600), scat_corr_plot.properties(width=600, height=600)).resolve_scale(color='independent'))

In [ ]:
year_dropdown_corr_line = alt.binding_select(options=year_options, name='year: ')
year_selection_corr_line = alt.selection_point(fields=['year'], bind=year_dropdown_corr_line, value='all')
feature_x_selection_corr_line = alt.selection_point(fields=['feature_x'], clear=False, name='feature_x_selection_corr_line', value='sea_brand_mel_spend')
feature_y_selection_corr_line = alt.selection_point(fields=['feature_y'], clear=False, name='feature_y_selection_corr_line', value='affiliate_mel_spend')


corr_line_base = alt.Chart(corr_df).encode(
    x='feature_x', y='feature_y'    
).mark_rect().encode(
    color=alt.condition(feature_x_selection_corr_line & feature_y_selection_corr_line, alt.value('pink'), 'corr')
).add_params(
    year_selection_corr_line, 
    feature_x_selection_corr_line,
    feature_y_selection_corr_line
).transform_filter(
    year_selection_corr_line
).properties(
    width=1200, height=600
)

corr_line_text = corr_line_base.mark_text().encode(
    text='corr_lab', 
    color=alt.condition(alt.datum.corr > 0.5, alt.value('white'), alt.value('black'))
)

corr_line_plot = corr_line_base + corr_line_text

melt_model_df = pd.melt(model_df, id_vars='date', value_vars=features_lst, var_name='feature')
line_corr_plot = alt.Chart(melt_model_df).transform_filter(
    'indexof(datum.feature, feature_x_selection_corr_line.feature_x) !== -1 | indexof(datum.feature, feature_y_selection_corr_line.feature_y) !== -1'
).mark_line().encode(
    x='date', y='value', color='feature'
).properties(
    width=1200, height=600
)

display(alt.hconcat(corr_line_plot.properties(width=600, height=600), line_corr_plot.properties(width=600, height=600)).resolve_scale(color='independent'))

In [ ]:
year_dropdown = alt.binding_select(options=year_options, name='year: ')
year_selection = alt.selection_point(fields=['year'], bind=year_dropdown, value='all')

x_feature_dropdown = alt.binding_select(options=features_lst, name='x_feature: ')
x_feature_param = alt.param(value='affiliate_mel_spend', bind=x_feature_dropdown)

y_feature_dropdown = alt.binding_select(options=features_lst, name='y_feature: ')
y_feature_param = alt.param(value='sea_brand_mel_spend', bind=y_feature_dropdown)


scatter_plot = alt.Chart(copy_model_df).mark_point().encode(
    x=alt.X('x:Q', title=''), y=alt.Y('y:Q', title='')
).add_params(
    year_selection, 
    x_feature_param, 
    y_feature_param
).transform_filter(
    year_selection
).transform_calculate(
    x=f'datum[{x_feature_param.name}]', 
    y=f'datum[{y_feature_param.name}]'
).properties(
    width=1200, height=600
)
display(scatter_plot)

In [ ]:
melt_model_df = pd.melt(model_df, id_vars='date', value_vars=features_lst, var_name='feature')

legend_selection = alt.selection_point(fields=['feature'], bind='legend', value=[{'feature': 'affiliate_mel_spend'}, {'feature': 'sea_brand_mel_spend'}])

line_custom_scale = alt.Scale(domain=features_lst)

line_plot = alt.Chart(melt_model_df, title='Not Scaled Features (Shift + Click Legend to select multiple)').mark_line().encode(
    x='date', y='value', color=alt.Color('feature', scale=line_custom_scale), opacity=alt.condition(legend_selection, alt.value(1), alt.value(0))
).add_params(
    legend_selection
).transform_filter(
    legend_selection
).properties(
    width=1200, height=600
)
display(line_plot)

In [ ]:
ss = StandardScaler(with_mean=True, with_std=True)
ss_model_df = pd.concat([model_df['date'], pd.DataFrame(ss.fit_transform(model_df[features_lst + ['n_conversion']]), columns=features_lst + ['n_conversion'])], axis=1)
ss_melt_model_df = pd.melt(ss_model_df, id_vars='date', value_vars=features_lst + ['n_conversion'], var_name='feature', value_name='scaled_value')

ss_legend_selection = alt.selection_point(fields=['feature'], bind='legend', value=[{'feature': 'n_conversion'}, {'feature': 'sea_brand_mel_spend'}])

ss_line_custom_scale = alt.Scale(domain=features_lst + ['n_conversion'])

ss_line_plot = alt.Chart(ss_melt_model_df, title='Scaled Features (Shift + Click Legend to select multiple)').mark_line().encode(
    x='date', y='scaled_value', color=alt.Color('feature', scale=ss_line_custom_scale), opacity=alt.condition(ss_legend_selection, alt.value(1), alt.value(0))
).add_params(
    ss_legend_selection
).transform_filter(
    ss_legend_selection
).properties(
    width=1200, height=600
)
display(ss_line_plot)